In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [2]:
# Getting table from Wikipedia Page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html (url)
df_table = tables[0]


# Drop row when Borough value is 'Not assigned'
table_edited = df_table[df_table.Borough != 'Not assigned']

# Join row if has more than one Neighbourhood
postcode_table = table_edited.groupby( ['Postcode', 'Borough'], sort=False).agg(','.join)

# Reset index
postcode_table.reset_index(inplace = True)

# Replace value of col Neighbourhood if = 'Not assigned' with Borough value
postcode_table['Neighbourhood'] = np.where(postcode_table['Neighbourhood'] == 'Not assigned', postcode_table['Borough'], postcode_table['Neighbourhood'])

postcode_table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [3]:
postcode_table.shape

(103, 3)

In [9]:
#Get 'Latitude' and 'Longitude' for each postcode from csv file
coordinates = pd.read_csv("http://cocl.us/Geospatial_data") 
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
#Add new colomns of 'Latitude' and 'Longitude' for each post code
merged_table = pd.merge(left=postcode_table,right=coordinates, how='left', left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1) 
merged_table.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [21]:
merged_table.shape

(103, 5)